In [24]:
OLLAMA_BASE_URL="https://61d7-107-20-19-92.ngrok-free.app"
OLLAMA_MODEL="llama3"

# Setup LLM

In [25]:
import json
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOllama(model=OLLAMA_MODEL, base_url=OLLAMA_BASE_URL, format="json")

## Role play Prompt

In [26]:
system_prompt = """ You are an expert at generating role play conversations in {language} \
Given an input phrase in {language}, you generate a role play conversation in {language}\
between two participants with the following in JSON format: \

Background: A short description of the a role play scenario in {language}. Maximum {max_words} words. \
<Name of First Participant>: Statement made to the second particpant in {language}. Maximum {max_words} words\
<Name of Second participant>: Response to the first participant in {language}, Maximum {max_words} words. \
"""

human_prompt = "phrase: {phrase}"

role_play_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", human_prompt),
    ]
)


async def generate_roleplay(language: str, phrase: str, max_words: int):
    all_chunks = []
    display_frequency = 20

    chain = role_play_prompt | llm | StrOutputParser()

    async for chunk in chain.astream({"language": language, 
                                    "phrase": phrase,
                                    "max_words": max_words
                                    }):
        all_chunks.append(chunk)
        # print just to confirm that llm is working
        if len(all_chunks) % display_frequency == 0:
            print(chunk)
    
    return "".join(all_chunks)


In [27]:
language = "German"
phrase = "unglaublich"
max_words = 25

response = await generate_roleplay(language, phrase, max_words)
print("******Generated Role Play ******")
print(response)

ise
 der
 Berlin
bah
indr
******Generated Role Play ******
{ "Background": "Eine Gruppe von Freunden plaudert über ihre letzte Reise nach Berlin.",
"Max Mustermann": "Ich bin immer noch geschockt darüber, dass der Hauptbahnhof so unglaublich groß ist! Ich dachte, ich kenne Berlin, aber das war ein Schock für mich.",
"Laura Mayer": "Ja, der Hauptbahnhof ist tatsächlich beeindruckend. Aber was hat dich am meisten beeindruckt an deiner Reise? Die Museen oder die Straßen?" }


In [30]:
response_json = json.loads(response)
response_json

{'Background': 'Eine Gruppe von Freunden plaudert über ihre letzte Reise nach Berlin.',
 'Max Mustermann': 'Ich bin immer noch geschockt darüber, dass der Hauptbahnhof so unglaublich groß ist! Ich dachte, ich kenne Berlin, aber das war ein Schock für mich.',
 'Laura Mayer': 'Ja, der Hauptbahnhof ist tatsächlich beeindruckend. Aber was hat dich am meisten beeindruckt an deiner Reise? Die Museen oder die Straßen?'}

# Translate Roleplay Prompt to English

In [34]:
system_prompt = """ You are an expert Translator from any language to English \
Given a JSON dictionary, return a JSON with translated dictionary values.
"""
human_prompt = """
dictionary: {dictionary}
"""

translation_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", human_prompt),
    ]
)


async def generate_translation(dictionary: dict):
    all_chunks = []
    display_frequency = 20

    chain = translation_prompt | llm | StrOutputParser()

    async for chunk in chain.astream({"dictionary": dictionary}):
        all_chunks.append(chunk)
        # print just to confirm that llm is working
        if len(all_chunks) % display_frequency == 0:
            print(chunk)
    
    return "".join(all_chunks)

In [35]:
translated_response = await generate_translation(response)
print(translated_response)

Max
 I
 the
 streets
{ "Background": "A group of friends are chatting about their latest trip to Berlin.",
"Max Mustermann": "I'm still shocked that the main station is so incredibly huge! I thought I knew Berlin, but it was a shock for me.", 
"Laura Mayer": "Yes, the main station is indeed impressive. But what impressed you most on your trip? The museums or the streets?" }


In [36]:
translated_response_json = json.loads(translated_response)
translated_response_json

{'Background': 'A group of friends are chatting about their latest trip to Berlin.',
 'Max Mustermann': "I'm still shocked that the main station is so incredibly huge! I thought I knew Berlin, but it was a shock for me.",
 'Laura Mayer': 'Yes, the main station is indeed impressive. But what impressed you most on your trip? The museums or the streets?'}